In [1]:
username = "Alex"
pathtorepo = "C:\\Users\\" *username *  "\\Desktop\\"
using Pkg
Pkg.activate(pathtorepo * "dynamical-systems\\env\\integrate\\")
include(pathtorepo * "dynamical-systems\\system.jl")


using StaticArrays, DifferentialEquations, DynamicalSystems, JLD, LinearSolve
E, x, y  = -40..40, -5..5,  -5..5
box = E × x × y
using GLMakie, LinearAlgebra

  Activating project at `C:\Users\Alex\Desktop\dynamical-systems\env\integrate`


In [2]:
cd("C:\\Users\\" *username *  "\\Desktop\\dynamical-systems\\Tsodyks Markram\\Levanova\\3 набор параметров\\Сопоставление с матконт\\файлы matlab")
I0_hom = load("I0_hom_hom.jld")["data"]
u0_hom = load("U0_hom_hom.jld")["data"]
I0_hom = I0_hom[:]
u0_hom = u0_hom[:];

In [3]:
const τ = 0.013;  const τD = 0.07993;  const τy = 3.3;  const J = 3.07;  const β = 0.300
const xthr = 0.75; const ythr = 0.4; const α = 1.58; const ΔU0 = 0.305

0.305

In [62]:
index_hom = 15
I0 = I0_hom[index_hom]; U0 = u0_hom[index_hom]
p = SA[α, τ, τD, τy, J, xthr, ythr, U0, ΔU0, β, I0]
u0 = [11.325905642223786, 0.6594706953104683, 0.4863175026548461];

In [63]:
ds = CoupledODEs(TM, u0, p)
fp, _, _ = fixedpoints(ds, box, jacob_TM_)
ϵ_sep = 1e-6
Ju0 = jacob_TM_(fp[1], p, 0)

eigen_val_vec = eigen(Ju0)
eigen_vectors = eigen_val_vec.vectors
v1 = real(eigen_vectors[:, 1])
dot_on_sep_stable = fp[1] + v1*ϵ_sep

3-element SVector{3, Float64} with indices SOneTo(3):
 8.345028421113797
 0.7384936001924872
 0.43829361169443143

In [127]:
t = 1000.0; tspanback = [0.0, -t]
prob = ODEProblem(TM, dot_on_sep_stable, tspanback, p)
sol = solve(prob, Rodas5P(), abstol = 1e-9, reltol = 1e-9)

┌ Warning: Instability detected. Aborting
└ @ SciMLBase C:\Users\Alex\.julia\packages\SciMLBase\KcGs1\src\integrator_interface.jl:606


retcode: Unstable
Interpolation: specialized 4rd order "free" stiffness-aware interpolation
t: 9688-element Vector{Float64}:
   0.0
  -0.08234278938908962
  -0.28667969438483276
  -0.37856457272165706
  -0.524611783116551
  -0.6180403146264019
  -0.7469878464950414
  -0.8370330815756925
  -0.9463340399894501
  -1.0303109533863775
   ⋮
 -13.917380907635469
 -13.918383314962739
 -13.91938572229001
 -13.92038812961728
 -13.92139053694455
 -13.92239294427182
 -13.92339535159909
 -13.92439775892636
 -13.925400166253631
u: 9688-element Vector{SVector{3, Float64}}:
 [8.345028421113797, 0.7384936001924872, 0.43829361169443143]
 [8.345029112996892, 0.7384935662474639, 0.43829361994649424]
 [8.345033661997347, 0.7384933430655826, 0.43829367420223986]
 [8.345038648878102, 0.7384930984012834, 0.43829373368045293]
 [8.345055977583707, 0.7384922482336865, 0.4382939403579087]
 [8.345079309227208, 0.7384911035696552, 0.43829421862953344]
 [8.345145735521033, 0.7384878447523283, 0.4382950108680305]
 [8

In [109]:
sol.interp

OrdinaryDiffEq.InterpolationData{ODEFunction{false, SciMLBase.AutoSpecialize, typeof(TM), UniformScaling{Bool}, Nothing, Nothing, Nothing, Nothing, Nothing, Nothing, Nothing, Nothing, Nothing, Nothing, Nothing, Nothing, Nothing, typeof(SciMLBase.DEFAULT_OBSERVED), Nothing, Nothing}, Vector{SVector{3, Float64}}, Vector{Float64}, Vector{Vector{SVector{3, Float64}}}, OrdinaryDiffEq.Rosenbrock5ConstantCache{SciMLBase.TimeDerivativeWrapper{ODEFunction{false, SciMLBase.AutoSpecialize, typeof(TM), UniformScaling{Bool}, Nothing, Nothing, Nothing, Nothing, Nothing, Nothing, Nothing, Nothing, Nothing, Nothing, Nothing, Nothing, Nothing, typeof(SciMLBase.DEFAULT_OBSERVED), Nothing, Nothing}, SVector{3, Float64}, SVector{11, Float64}}, SciMLBase.UDerivativeWrapper{ODEFunction{false, SciMLBase.AutoSpecialize, typeof(TM), UniformScaling{Bool}, Nothing, Nothing, Nothing, Nothing, Nothing, Nothing, Nothing, Nothing, Nothing, Nothing, Nothing, Nothing, Nothing, typeof(SciMLBase.DEFAULT_OBSERVED), Nothi

In [21]:
homcurve = range(1, 150, step = 1)

1:1:150

In [ ]:
for index in homcurve
    
    I0 = I0_hom[index]; U0 = u0_hom[index]
    p = SA[α, τ, τD, τy, J, xthr, ythr, U0, ΔU0, β, I0]

    println("I0: $(I0); U0: $(U0)")

    ds = CoupledODEs(TM, u0, p)
    fp, _, _ = fixedpoints(ds, box, jacob_TM_)
    ϵ_sep = 1e-6
    Ju0 = jacob_TM_(fp[1], p, 0)

    eigen_val_vec = eigen(Ju0)
    eigen_vectors = eigen_val_vec.vectors
    v1 = real(eigen_vectors[:, 1])
    u0 = fp[1] + v1*ϵ_sep

    prob = ODEProblem(TM, u0, tspanback, p)
    sol = solve(prob, Rodas5(), adaptive = true, abstol = 1e-10, reltol = 1e-10, verbose = false)
    println("Index: $(sol.retcode)")
    println("----------------------")
end

In [130]:
ts, tf = 1, 700

idx, idy, idz = 2, 3, 1
dis  = 60;
    
f = Figure(resolution = (700, 700))
ax = LScene(f[1, 1], show_axis = true)
scale!(ax.scene, 50, 50, 1)

lines!(sol[idx, ts:tf], sol[idy, tsb:tf], sol[idz, ts:tf], linewidth = 1.0, color = :black)

for (i, ep) in enumerate(fp)
    scatter!(ep[idx], ep[idy], ep[idz], color = :blue, markersize = 8)
end

display(GLMakie.Screen(), f)

GLMakie.Screen(...)